In [ ]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit

In [ ]:
#pref = "DUMP_run9441_laseronly.csv"
#postf = "DUMP_run9441_lasercosmics.csv"

pref = "DUMP_run12014_laseronly.csv"
postf = "DUMP_run12014_lasercosmics.csv"

dpre = pd.read_csv(pref)
dpost = pd.read_csv(postf)

In [ ]:
dpre.head()

In [ ]:
SKIP = True

if(not SKIP):
    for c in np.unique(dpre.channel_id.to_numpy()):

        _selpre = (dpre.channel_id == c)
        _selpost = (dpost.channel_id == c)
        _selcorr = (corr.channel_id == c)

        fig = plt.figure(dpi=200)

        rmin=-10
        rmax=10
        r=(rmin,rmax)
        s=0.3
        b=int((rmax-rmin)/s)

        plt.hist(dpre[_selpre].residuals*1e3, bins=b, linewidth=2, range=r, histtype="step", label="PRE")
        plt.hist(dpost[_selpost].residuals*1e3, bins=b, linewidth=2, range=r, histtype="step", label="POST")

        plt.title("channel {}".format(c))
        plt.legend()
        plt.grid()
        plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

def fittime(y, t):
    if len(y) < 4 or len(t) < 4:
        print("Not enough data points for linear regression: y = {}, t = {}".format(y, t))
        return 0, 0

    # Define a linear model: t = intercept + slope * y
    def linear_model(x, slope, intercept):
        return intercept + slope * x

    try:
        popt, _ = curve_fit(linear_model, y, t)
        return popt[0], popt[1]
    except Exception as e:
        print(e) 
        
    print("Fitting did not converge for y = {}, t = {}".format(y, t))
    return 0, 0


def generate_plots(dpre, dpost, limit=10, pecut=150, output_pdf='plots.pdf'):
    # Create the union of unique (event, cryo, flash_id) tuples from both dataframes
    unique_tuples = pd.concat([dpre[['event','cryo','flash_id']], 
                               dpost[['event','cryo','flash_id']]]).drop_duplicates()

    try:
        run_val = dpre.run.iloc[0]
    except AttributeError:
        run_val = 'N/A'

    output_pdf = "run{}_".format(run_val) + output_pdf

    count = 0
    with PdfPages(output_pdf) as pdf:
        for tup in unique_tuples.itertuples(index=False):
            if count >= limit:
                break
            event_val, cryo_val, flash_val = tup

            # Filter for the current (event, cryo, flash) tuple
            epre = dpre[(dpre.event == event_val) & (dpre.cryo == cryo_val) & (dpre.flash_id == flash_val)]
            epost = dpost[(dpost.event == event_val) & (dpost.cryo == cryo_val) & (dpost.flash_id == flash_val)]
            
            # Skip if there are no entries in either dataframe for this tuple
            if epre.empty or epost.empty:
                continue

            # Apply pmt_pe cut
            _selpre = epre.pmt_pe > pecut
            _selpost = epost.pmt_pe > pecut
            if epre[_selpre].empty or epost[_selpost].empty:
                continue

            meanpre = epre.groupby(["run", "event", "cryo", "flash_id", "pmt_y"]).apply( 
                            lambda x : pd.Series( {
                                                    "mean_time" : np.mean(x.pmt_time),
                                                    "weight_mean_time" : np.average(x.pmt_time, weights=x.pmt_pe), 
                                                    "error_mean_time": np.std(x.pmt_time) / np.sqrt(len(x.pmt_time)),
                                                    }) ).reset_index()
            meanpost = epost.groupby(["run", "event", "cryo", "flash_id", "pmt_y"]).apply( 
                            lambda x : pd.Series( {
                                                    "mean_time" : np.mean(x.pmt_time),
                                                    "weight_mean_time" : np.average(x.pmt_time, weights=x.pmt_pe), 
                                                    "error_mean_time": np.std(x.pmt_time) / np.sqrt(len(x.pmt_time)),
                                                    }) ).reset_index()

            mpre0, ipre0 = fittime(meanpre.pmt_y.to_numpy(), meanpre.mean_time.to_numpy())
            mpost0, ipost0 = fittime(meanpost.pmt_y.to_numpy(), meanpost.mean_time.to_numpy())

            #print(mpre0, ipre0, mpost0, ipost0)

            # Create a new figure for this tuple
            fig, ax = plt.subplots(figsize=(10, 4.3), dpi=200)
            
            # Scatter plots for PRE and POST
            ax.scatter(epre[_selpre].pmt_y, epre[_selpre].pmt_time, s=20, label='PRE')
            ax.scatter(epost[_selpost].pmt_y, epost[_selpost].pmt_time, s=20, label='POST')

            # Scatter plots for PRE and POST
            ax.errorbar( x=meanpre.pmt_y, y=meanpre.mean_time, yerr=meanpre.error_mean_time, marker='o', elinewidth=2.0, lw=0, label='PRE - mean time', color='black' )
            ax.errorbar( x=meanpost.pmt_y, y=meanpost.mean_time, yerr=meanpost.error_mean_time, marker='o', elinewidth=2.0, lw=0, label='PRE - mean time', color='gray' )

            # Use the first valid row for the linear fit parameters
            mpre = epre[_selpre].slope.values[0]
            ipre = epre[_selpre].intercept.values[0]
            mpost = epost[_selpost].slope.values[0]
            ipost = epost[_selpost].intercept.values[0]

            x_vals = np.linspace(-150, 100, 250)
            ax.plot(x_vals, x_vals * mpre + ipre, lw=2.0, color='red', label='PRE fit')
            ax.plot(x_vals, x_vals * mpost + ipost, lw=2.0, color='magenta', label='POST fit')

            ax.plot(x_vals, x_vals * mpre0 + ipre0, lw=2.0, color='blue', label='PRE fit0')
            ax.plot(x_vals, x_vals * mpost0 + ipost0, lw=2.0, color='purple', label='POST fit0')

            ax.set_xlabel("PMT Y position [cm]", fontsize=14)
            ax.set_ylabel("PMT first photon time [us]", fontsize=14)
            ax.grid(alpha=0.5, linestyle='dashed')
            ax.legend(loc='lower left', fontsize=14)

            # Use the run value from the dataframe if available, otherwise mark as 'N/A'
            watermark = r'$\mathbf{ICARUS}\,$ Data' + ' - Run {}, Event {}, Module {}, Flash ID {}'.format(
                run_val, event_val, cryo_val, flash_val)
            ax.text(0.30, 1.05, watermark, fontsize=12, color='black', alpha=1,
                    ha='left', va='center', transform=ax.transAxes)

            fig.tight_layout()
            pdf.savefig(fig)
            plt.close(fig)
            print("Event {}, cryo {}, flash {}: mpre {:.6f} mpost {:.6f}".format(event_val, cryo_val, flash_val,mpre,mpost))
            print(mpre0, mpost0)
            count += 1

# Example usage:
# generate_plots(dpre, dpost, limit=5, pecut=150, output_pdf='my_plots.pdf')


In [ ]:
generate_plots(dpre, dpost, limit=200, pecut=150, output_pdf='plots.pdf')


In [ ]:
EVENT=50837
CRYO=0
FLASH=2
PECUT=150

# Keep only the reference event
epre = dpre[(dpre.event==EVENT) & (dpre.cryo==CRYO) & (dpre.flash_id==FLASH)]
epost = dpost[(dpost.event==EVENT) & (dpost.cryo==CRYO) & (dpost.flash_id==FLASH)]

print("Number of hits in this flash: {} {}".format(len(epre),len(epost)))

In [ ]:
fig, ax = plt.subplots( 1,1, figsize=(10, 4.3), dpi=200)

_selpre = epre.pmt_pe > PECUT
_selpost = epost.pmt_pe > PECUT

outepre = ax.scatter( epre[_selpre].pmt_y, epre[_selpre].pmt_time, s=20, label='PRE' ) #jet
outepost = ax.scatter( epost[_selpost].pmt_y, epost[_selpost].pmt_time, s=20, label='POST' )

mpre = epre[_selpre].slope.values[0]
ipre = epre[_selpre].intercept.values[0]

mpost = epost[_selpost].slope.values[0]
ipost = epost[_selpost].intercept.values[0]

print("{:.2e}, {:.2e}".format(mpre, mpost))

ax.plot( np.linspace(-150, 100),  np.linspace(-150, 100)*mpre+ipre, lw=2.0, color='red', label='PRE fit' )
ax.plot( np.linspace(-150, 100),  np.linspace(-150, 100)*mpost+ipost, lw=2.0, color='magenta', label='POST fit' )

ax.set_ylabel("PMT first photon time [us]", fontsize=14)
ax.set_xlabel("PMT Y position [cm]", fontsize=14)
ax.grid(alpha=0.5, linestyle='dashed')
ax.legend()
ax.legend(loc='lower left',fontsize=14)

watermark = r'$\mathbf{ICARUS}\,$ Data' +' - Run {}, Event {}, Module {}, Flash ID {}'.format(
            RUN, EVENT, CRYO, FLASH)
ax.text(0.30, 1.05, watermark, fontsize=12, color='black', alpha=1,
         ha='left', va='center', transform=ax.transAxes)

fig.tight_layout()
plt.show()